In [ ]:
!pip install qdrant-client

In [3]:
import pandas as pd
from pathlib import Path
from qdrant_client import models, QdrantClient
from nlp_models.multi_task_model.mtl import MTLInference

DATA_FOLDER = Path('../../data/0_external/google-quest-challenge')
MODEL_DIR = Path('../../models/multi-task-model/')

In [4]:
output_dir = MODEL_DIR / 'multi-task-model-finetuned-classification-layer-20230609'
tokenizer_dir = output_dir / 'tokenizer'
model_file = output_dir / 'mtl.bin'

mtl = MTLInference(tokenizer_dir, model_file, pretrained_model=False)

In [5]:
df_train = pd.read_csv(DATA_FOLDER / 'train.csv')
label_dict = dict([(k,v) for k, v in enumerate(df_train.category.unique())])

In [6]:
qdrant = QdrantClient(':memory:')
qdrant.recreate_collection(
    collection_name='qb',
    vectors_config=models.VectorParams(
        size=mtl.mtl_model.base_model.config.hidden_size,
        distance=models.Distance.DOT
    )
)

True

In [7]:
qdrant.upload_records(
    collection_name='qb',
    records=[
        models.Record(
            id=idx,
            vector=mtl.predict(rec['question_title'])[1].squeeze().tolist(),
            payload=rec
        ) for idx, rec in df_train.iterrows()
    ]
)

In [8]:
query='What helmet to use for biking in winter?'
matches = qdrant.search(
    collection_name='qb',
    query_vector=mtl.predict(query)[1].squeeze().tolist(),
    limit=3
)
matches

[ScoredPoint(id=4083, version=0, score=0.8482221290196361, payload={'qa_id': 6518, 'question_title': 'Using a ski helmet for winter biking', 'question_body': 'I am curious if anyone uses a skiing helmet for winter biking? Would this be safe? Are skiing helmets designed to protect you from the same kind of accidents that you would have on a bike?  \n\nMy thinking is that a ski helmet would:\n\n\nHelp to keep your ears warm\nWould fit well with ski goggles (which fit poorly with my bike helmet)\nNot have air vents in it which make you cold in the winter\n\n', 'question_user_name': 'sixtyfootersdude', 'question_user_page': 'https://bicycles.stackexchange.com/users/134', 'answer': "I wear a Bern Brentwood with a winter liner in cold weather. It's a certified bike helmet, but designed more like a ski helmet. The winter liner does a great job of keeping my ears warm without wearing any other protection, but doesn't block traffic noise. It is vented, but not well enough that it makes my head 

In [ ]:
query = 'what helmet to use for biking in winter?'
outputs = mtl.predict(query)
matches = qdrant.search(
    collection_name='qb',
    query_vector=outputs[1].squeeze().tolist(),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key='category',
                match=models.MatchValue(
                    value=label_dict[outputs[0].squeeze().gt(0.5).nonzero().item()]
                    )
            )
        ]
    )
)